In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import iplot

In [20]:
# Read the grant csv file
grants_to_people = pd.read_csv('grants_to_people.csv')

In [21]:
# Extract unique nodes
grants = grants_to_people['from'].unique()
people = grants_to_people['to'].unique()
nodes = list(grants) + list(people)

In [5]:
# print(nodes)

In [22]:
# Create edges
edge_x = []
edge_y = []
for _, row in grants_to_people.iterrows():
    grant_index = list(nodes).index(row['from'])
    person_index = list(nodes).index(row['to'])
    edge_x.extend([grant_index, person_index, None])
    edge_y.extend([0, 1, None]) # Dummy Y values, as Y will not vary


In [23]:
# Create a trace for edges
# --> create the lines for each edge in plotly fig
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

In [24]:
# Assign colors based on node type
node_colors = []  # List to hold the color for each node
for node in nodes:
    if node in grants:
        node_colors.append('yellow')  # Color for grant nodes
    else:
        node_colors.append('blue')  # Color for person nodes

In [25]:
# Create a trace for nodes
# --> create the node for each node in plotly fig
node_x = []
node_y = []
for node in nodes:
    node_x.append(list(nodes).index(node))
    node_y.append(0 if node in grants else 1) # Assign Y based on type

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        color=node_colors,
        size=10,
        line_width=0.5))


In [26]:
# Add text to nodes
node_text = []
for node in nodes:
    node_text.append(node)
node_trace.text = node_text

In [27]:
# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph of Grants to People',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )


In [28]:
# Show the figure
fig.show()

In [30]:
import plotly

plotly.offline.plot(fig, filename='grant_application_graph.html')

'grant_application_graph.html'

In [13]:
import pandas as pd
import plotly.graph_objects as go

# Load the data from CSV files
df_grants_to_people = pd.read_csv('grants_to_people.csv')
df_people_to_people = pd.read_csv('people_to_people.csv')

# Extract unique nodes for grants and initial set of people
grants = df_grants_to_people['from'].unique()
people = pd.concat([df_grants_to_people['to'], df_people_to_people['from'], df_people_to_people['to']]).unique()

# Combine into a single list of unique nodes
nodes = list(set(grants).union(set(people)))

# Initialize the list of edges
edges = []

# Add grants-to-people edges
for _, row in df_grants_to_people.iterrows():
    edges.append((row['from'], row['to']))

# Add people-to-people edges
for _, row in df_people_to_people.iterrows():
    edges.append((row['from'], row['to']))

# Separate edges by type
edges_gp = []  # Edges connecting grants to people
edges_pp = []  # Edges connecting people to people

for edge in edges:
    if edge[0] in grants:
        edges_gp.append(edge)
    else:
        edges_pp.append(edge)

# Prepare Plotly traces for edges
edge_traces = []

# Trace for grant-to-person edges (orange)
for edge in edges_gp:
    x0, x1 = [nodes.index(edge[0]), nodes.index(edge[1])]
    y0, y1 = [0, 1]  # Simplified for demonstration
    edge_traces.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines',
                                  line=dict(width=0.5, color='orange'), hoverinfo='none'))

# Trace for people-to-people edges (light blue)
for edge in edges_pp:
    x0, x1 = [nodes.index(edge[0]), nodes.index(edge[1])]
    y0, y1 = [1, 1]  # Adjust based on your layout needs
    edge_traces.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines',
                                  line=dict(width=0.5, color='lightblue'), hoverinfo='none'))


# Node trace with colors (grants: yellow, people: blue)
node_x = [nodes.index(node) for node in nodes]
node_y = [0 if node in grants else 1 for node in nodes]  # Simplified positioning
node_colors = ['yellow' if node in grants else 'blue' for node in nodes]

node_trace = go.Scatter(x=node_x, y=node_y, mode='markers', hoverinfo='text', marker=dict(color=node_colors, size=10, line=dict(width=0.5)))

# Node labels as hover text
node_trace.text = nodes

# Create the figure
fig = go.Figure(data=edge_traces + [node_trace], layout=go.Layout(
    title='Network graph with differentiated edge types',
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    annotations=[dict(text="Note: Orange edges connect grants and people; light blue edges connect people.",
                      showarrow=False, xref="paper", yref="paper", x=0.005, y=-0.002)],
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.show()


In [14]:
import math

# Load the people-to-people data
df_people_to_people = pd.read_csv('people_to_people.csv')

# Get the unique list of people from the edges
unique_people = pd.concat([df_people_to_people['from'], df_people_to_people['to']]).unique()

# Calculate positions in a circle
circle_radius = 1  # Can be adjusted for better spacing
angle_step = 2 * math.pi / len(unique_people)
node_x = [circle_radius * math.cos(i * angle_step) for i in range(len(unique_people))]
node_y = [circle_radius * math.sin(i * angle_step) for i in range(len(unique_people))]

# Map each person to their position
node_pos = {person: (x, y) for person, x, y in zip(unique_people, node_x, node_y)}

# Prepare the edge traces
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=2, color='lightblue'),
    hoverinfo='none',
    mode='lines')

for _, row in df_people_to_people.iterrows():
    x0, y0 = node_pos[row['from']]
    x1, y1 = node_pos[row['to']]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Prepare the node trace
node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    text=unique_people,
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        color='blue',
        size=10,
        line=dict(width=2)))

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='People-to-People Network Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    xaxis_range=[-1.5, 1.5],
                    yaxis_range=[-1.5, 1.5]
                ))

fig.update_layout(showlegend=False)
fig.show()
